In [1]:
import ctypes
import time
import os
import multiprocessing

In [2]:
_CCnt = ctypes.CDLL('./CCnt.so')

In [3]:
#initialize counter on both CPUs

# Launch process on CPU0
p1 = multiprocessing.Process(target=_CCnt.init, args=(1,0))
os.system("taskset -p -c {} {}".format(0, p1.pid))
p1.start() # start the process

# Launch process on CPU1
p2 = multiprocessing.Process(target=_CCnt.init, args=(1,0))
os.system("taskset -p -c {} {}".format(1, p2.pid))
p2.start() # start the process

p1.join()
p2.join()
print('CPU Counters initialized')

CPU Counters initialized


In [10]:
signed = _CCnt.getCC()
unsigned = signed
if unsigned < 0:
    unsigned += 2**32
print(signed)
print(unsigned)

422891648
422891648


In [8]:
t1 = 4294967295
t2 = 1
t3 = t2-t1
t3 += 2**32
print(t3)

2


In [11]:
def recur_fibo(n):
   if n <= 1:
       return n
   else:
       return(recur_fibo(n-1) + recur_fibo(n-2))


In [119]:
def timed_fib(n):
    time.sleep(0.1) #delay is needed to allow this to be assigned to a specific core after starting the process
    
    _CCnt.init(1,0)
    
    t_before = time.time()
    cc_before = _CCnt.getCC()

    recur_fibo(n)
        
    cc_after = _CCnt.getCC()
    t_after = time.time()

    #convert to unsigned
    if cc_after < 0:
        cc_after += 2**32
    if cc_before < 0:
        cc_before += 2**32
        
    elapsed_time = t_after-t_before
    num_cycles = cc_after-cc_before
    
    #account for possibility of overflow
    if num_cycles < 0:
        num_cycles += 2**32
    
    print(cc_before)
    print(cc_after)
    print(elapsed_time)
    print(num_cycles)
    
    print('Time based on clock cycles')
    print(num_cycles/650000000)

In [124]:
n = 25
#CPU = '0x01'
CPU = 0

#convert to bit mask
#if CPU == 0:
#    CPU = '0x01'
#else:
#    CPU = '0x02'


p_fib = multiprocessing.Process(target=timed_fib, args=(n,))
#print(p_fib.pid)
#print(str)
#os.system("taskset -p -c {} {}".format(CPU, p_fib.pid))
#os.system("taskset -p 0x02 {}".format(p_fib.pid))
#os.system("taskset -p {} {}".format(CPU,p_fib.pid))
#os.system("taskset -p 0x01 {}".format(p_fib.pid))
#os.system(str)
#print(p_fib.pid)
p_fib.start()
os.system("taskset -pc {} {}".format(CPU,p_fib.pid))
#print(p_fib.pid)

p_fib.join()

26869
215866277
0.3321073055267334
215839408
Time based on clock cycles
0.3320606276923077


In [78]:
timed_fib(28)

2650201147
3564701272
1.4074559211730957
914500125
Time based on clock cycles
1.4069232692307692


In [88]:
#test with sleep
def timed_sleep(n):
    t_before = time.time()
    cc_before = _CCnt.getCC()

    time.sleep(n)
        
    cc_after = _CCnt.getCC()
    t_after = time.time()

    #convert to unsigned
    if cc_after < 0:
        cc_after += 2**32
    if cc_before < 0:
        cc_before += 2**32
        
    elapsed_time = t_after-t_before
    num_cycles = cc_after-cc_before
    
    #account for possibility of overflow
    if num_cycles < 0:
        num_cycles += 2**32
        
    print(elapsed_time)
    print(num_cycles)
    
    print('Time based on clock cycles')
    print(num_cycles/650000000)


In [100]:
timed_sleep(0.01)

0.010259866714477539
677229963
Time based on clock cycles
1.0418922507692308
